In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()    
        self.layer_1 = nn.Sequential(
            nn.Linear(1, 1)            
        )      

    def forward(self, x):                
        logits = self.layer_1(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)

In [ ]:
print(model)

NeuralNetwork(
  (layer_1): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
  )
)


In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [ ]:
X_train = df['Celsius']
Y_train = df['Fahrenheit']

In [ ]:
train_target = torch.tensor(Y_train.values.astype(np.float32))
train = torch.tensor(X_train.values.astype(np.float32)) 
train_tensor = TensorDataset(train, train_target) 
train_loader = DataLoader(dataset=train_tensor, batch_size=1, shuffle=True)

for X, y in train_loader:
    print("Shape of X: ", X.shape, X.dtype)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X:  torch.Size([1]) torch.float32
Shape of y:  torch.Size([1]) torch.float32


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)        

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() # set the gradients to zero before backpropragation
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)            
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
epochs = 450

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)

print("Done!")

Epoch 1
-------------------------------
loss: 6654.540039  [    0/   30]
Epoch 2
-------------------------------
loss: 7745.439453  [    0/   30]
Epoch 3
-------------------------------
loss: 960.685608  [    0/   30]
Epoch 4
-------------------------------
loss: 1119.469482  [    0/   30]
Epoch 5
-------------------------------
loss: 1372.439575  [    0/   30]
Epoch 6
-------------------------------
loss: 67.671875  [    0/   30]
Epoch 7
-------------------------------
loss: 5169.294434  [    0/   30]
Epoch 8
-------------------------------
loss: 1001.439880  [    0/   30]
Epoch 9
-------------------------------
loss: 1309.217651  [    0/   30]
Epoch 10
-------------------------------
loss: 778.029297  [    0/   30]
Epoch 11
-------------------------------
loss: 748.916077  [    0/   30]
Epoch 12
-------------------------------
loss: 656.428711  [    0/   30]
Epoch 13
-------------------------------
loss: 731.711731  [    0/   30]
Epoch 14
-------------------------------
loss: 739.039

In [ ]:
def c_to_f(c):
  return 9/5 * c + 32

In [ ]:
Temp_C = [0, 15, 35] # 32, 59, 95

for c in Temp_C:
  with torch.no_grad():
    model.eval()
    Temp_F = model(torch.tensor([np.float32(c)]))  
    print(f'C {c} = F {np.around(Temp_F[0])}')

C 0 = F 32.0
C 15 = F 59.0
C 35 = F 95.0


In [ ]:
for c in Temp_C:
  print(f'C {c} = F {c_to_f(c)}')

C 0 = F 32.0
C 15 = F 59.0
C 35 = F 95.0


In [ ]:
s = torch.sum(model.layer_1[0].weight.data)
s # 9/5 = 1.8

tensor(1.8000)

In [ ]:
s = torch.sum(model.layer_1[0].bias.data)
s # 32

tensor(32.0000)